# Project 1 Submission

## Logic (or lack thereof) behind approach

Through some trials (refer to "Project 1 Trials and Notes" ipynb file), I found my ideal approach to find the top banks with the best money/time ratios. Through multiple trials, I found the ideal number of options for my approach to be between 150-200 banks; I went with 200 banks. Then I used random to go through the banks with np.random.randint to arbitrarily select banks. 

The program will go through banks and add them to a pathway whilst keeping track of time. Once the time hits just under 24 hours, the program "breaks" the current iteration and appends the path, the money gains, and the time spent to different lists. 

This occurs a near infinite number of times until the while loop is forced to stop via the time-lapse "if-statement" set up in the beginning of the loop. Finally, the program selects the path with the highest gains and returns it.

## Flaws

The biggest and most obvious flaw of this is that there is basically no repeatability, since the program goes through a list of 200 banks randomly and stopping just shy of 24 hours. The likelihood of getting the same answer twice in a row (given the 3 minute run-time restriction) is basically impossible. 

Theoretically, if there was no time limit, there would be no need to isolate the maximum money/hour as you could run all possible pathways ignoring processing time and always get the same (best possible) response. However, based on my various trials, I have come to find that the money value I get with this strategy hovers around $6,500,000. So while there's no real repeatability based on the restrictions set in place, there is some level of consistency.

In [1]:
import pandas as pd
import numpy as np
import time
import math

df = pd.read_csv("bank_data.csv")
df_arr = np.array(df)
R_Dist = []

for i in range(len(df_arr)):
    R_Dist.append(df_arr[i,3]/df_arr[i,4])

df['money/hr'] = R_Dist

ratio = sorted(range(len(R_Dist)), key = lambda sub: R_Dist[sub])[-200:]
data = np.array(df.iloc[ratio,:])

start = np.argmax(data[:,3])
attempts = []
pot = []
times = []
timeout = 180 ## time out of 180 seconds (the requirement of a maximum 3 minutes) to stop loop
start_time = time.time()

while True:
    current_time = time.time()
    elapsed_time = current_time - start_time
    if elapsed_time > timeout:
        ideal = attempts[int(pot.index(max(pot)))]
        profit = max(pot)
        best_time = times[int(pot.index(max(pot)))]
        break
    path = [start] ##starting point
    dur = data[start, 4] + (math.hypot(abs(data[start,1]), abs(data[start,2])))/30
    ##starting duration, assumes you start at the first point and time counts accordingly
    value = data[start,3] ##starting amount for money
    prev_target = start
    for i in range(1, len(data)):
        if dur < 23:
            next_target = np.random.randint(0, len(data)-1)
            if next_target not in path:
                dur -= (math.hypot(abs(data[prev_target,1]), abs(data[prev_target,2])))/30
                dur += sum([(math.hypot(abs(data[prev_target,1] - data[next_target,1]),
                                        abs(data[prev_target,2] - data[next_target,2])))/30,
                                data[next_target,4],
                           (math.hypot(abs(data[next_target,1]), abs(data[next_target,2])))/30])
                value += data[next_target,3]
                path.append(next_target)
                prev_target = next_target
        else:
            attempts.append(path)
            pot.append(value)
            times.append(dur)
            break
        
targets = []
for item in range(0, len(ideal)):
    targets.append(ratio[int(ideal[item])])

print(len(targets),"banks along the path:", targets, "for ${:,}".format(profit),"taking", best_time, "hours.")

120 banks along the path: [7265, 6022, 5610, 1424, 444, 6528, 209, 7764, 9908, 9401, 3186, 6317, 433, 7531, 1447, 2037, 6712, 1684, 4789, 8550, 9275, 5155, 1372, 2656, 865, 1733, 3297, 8703, 613, 4234, 1058, 781, 5622, 4627, 6535, 487, 1966, 7769, 4906, 7544, 9653, 4762, 3007, 1997, 4696, 6740, 4036, 9546, 1455, 6097, 2827, 2590, 8125, 7074, 2190, 5719, 6623, 2226, 6281, 7087, 3026, 7665, 8022, 2643, 507, 4987, 9881, 2729, 8169, 2928, 8966, 2194, 4293, 7772, 7595, 3516, 6254, 8375, 6339, 1397, 5296, 5725, 754, 3089, 7625, 8355, 8829, 299, 7583, 70, 5126, 2346, 7258, 3193, 7801, 9241, 4345, 9779, 7494, 8525, 7560, 8436, 9583, 4725, 5381, 5563, 6104, 8231, 6759, 5933, 5356, 2458, 3798, 9195, 8690, 8287, 7701, 8469, 3914, 6216] for $6,772,600.0 taking 23.15035924925477 hours.


In [2]:
def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

def check_solution(travel_list, df, speed=30.):
    """
    Check a bank robber algorithm solution
    
    input:
    ------
    travel_list: a list of integers
        Your solution to the bank robber problem
        Bank IDs are row indeces into the df passed
    df: pd.DataFrame
        The algorithm input formatted dataframe 
        (id,x_coordinate,y_coordinate,money,time (hr))
    speed: float
        The km/h speed of travel
    """
    t_remaining = 24.
    score = 0.
    assert len(travel_list) == len(set(travel_list)), (
        "Your travel list must have unique IDs!"
    )
    prev = travel_list[0]
    for e in travel_list:
        row = df.iloc[e]
        score += row['money']
        t_remaining -= row['time (hr)']
        dist = distance(
            row['x_coordinate'], row['y_coordinate'],
            df.iloc[prev]['x_coordinate'],
            df.iloc[prev]['y_coordinate'],
        )
        t_remaining -= dist / speed
        prev = e
    assert t_remaining >= 0, (
        f"Used more than 24h! Time left: {t_remaining}"
    )
    # still gotta get to (0, 0)
    dist = distance(row['x_coordinate'],row['y_coordinate'],0,0)
    final_t = t_remaining - (dist / speed)
    assert final_t >= 0, (
        f"Not enough time to get to helicopter!\n"
        f"Time left after last bank: {t_remaining}\n"
        f"Distance to (0,0) helipad: {dist}\n"
    )
    print(f"Time Remaining: {final_t}")
    return score

check_solution(targets, pd.read_csv("bank_data.csv"), speed = 30)

Time Remaining: 0.8496407507452443


6772600.0